# Northwind database
##### Jonathan Tyler

Northwind is a company that sells specialty foods. 

I was given a mandate to  "__do something with this database__" as my first project as a new hire. 

I was able to find a entity-relation diagram for the database but not much else. So to break down this task:
1. First I am going to have to explore the database myself to see the basic metrics of the company.
2. Then I will use this basic data to formulate some hypotheses concerning some underlying trends.
3. Finally I will attempt to test out these ideas to prove myself right or wrong.

## EDA

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session, sessionmaker

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from scipy.stats import ttest_1samp, ttest_ind
from scipy.stats import levene, shapiro, mannwhitneyu, kruskal

plt.style.use('ggplot')

Connecting the database enable to start looking at the data. I plan to compare the ERD to the database and get basic metric from the tables.

In [ ]:
#engine = create_engine('sqlite:///Northwind_large.sqlite', echo=True)
engine = create_engine('sqlite:///Northwind_small.sqlite', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

inspector = inspect(engine) #checking the ERD against the actual database
inspector.get_table_names()

  
So a quick check of the different tables align with the ERD. That is good. But I should be checking out the individual tables before calling them nevertheless, it is good pratice.
  
Now lets start checking out a few things:
- How much of what are we selling?
- Who are our main suppliers?
- What does our customer base look like?
- What is the geographical spread of our workforce?

Once we know these things, we will have a broad overview of the business. From there we will investigate any abnormalities or go splunking for underlying trends.
___
---
Now lets make a connection to the engine and make sure it works.

In [ ]:
con = engine.connect() #connecting the engine to be able to make queries

In [ ]:
q = '''''' #simple query get everything from [Order]
df_order = pd.read_sql_query(q, engine) #puts the information from the query into a dataframe
df_order.head()

Fantasic, now lets start flushing out answers to those inital questions

## How much of what are we selling?

Lets check the Product table and the Order Detail table

Looks like the table is ERD is wrong again. Some of the tables are incorrectly stated i.e. ProductID is just Id

In [ ]:
q='''SELECT p.ProductName, c.CategoryName, SUM(o.Quantity) num_ordered \
FROM Product p \
LEFT JOIN OrderDetail o ON o.ProductId = p.Id \
LEFT JOIN Category c ON c.Id = p.CategoryId \
GROUP BY p.ProductName ORDER BY num_ordered DESC'''
df1 = pd.read_sql_query(q, engine)
df1.head()

In [ ]:
#value_counts of the categories


## Who are our main suppliers?

In [ ]:
q = '''SELECT s.CompanyName, s.Region, COUNT(*) num_of_orders FROM [Order] o \
LEFT JOIN OrderDetail od ON o.Id = od.OrderId \
LEFT JOIN Product p ON od.ProductId = p.Id \
LEFT JOIN Supplier s ON p.SupplierId = s.Id \
GROUP BY s.CompanyName \
ORDER BY num_of_orders DESC'''

df2 = pd.read_sql_query(q, engine)
df2.head()

In [ ]:
#value_counts to see what region
#average number of orders

In [ ]:
q = #query of where our employees are using the Employee table

df2 = pd.read_sql_query(q, engine)
df2.head()

In [ ]:
#now check out where all the regions these people are from

# 2) Is there a difference in the money generated from sales between the North America office and the British Island office?

The employees of this company are based in two central locations. I want to know if one office is better at business than the other. The metric will be money generated (products ordered * price) with the differential based on one office or another.

$H_{o} =$ The employees based out of the British Island office generates the same amount revenue, per order, than the North American office.  
$H_{a} =$ The employees based out of the British Island office generates less revenue, per order, than the North American office.  
$\alpha = 0.05$

### 2-1 Inital Analysis


In [ ]:
q = '''SELECT od.OrderId, SUM(od.UnitPrice*od.Quantity) AS generated, e.Region AS employ_from \
FROM OrderDetail od \
LEFT JOIN [Order] o ON od.OrderId = o.Id \
LEFT JOIN Employee e ON o.EmployeeId = e.Id
GROUP BY od.OrderId'''

df = pd.read_sql_query(q, engine)
df.head(10)

Looks like we have all 830 orders. Great.

In [ ]:
#Find group by the employ_from and then describe the generated amount

The average made by both offices are close to each other but there is far fewer sales made by the British office. Lets take a quick look at the distribution for both, but look at the above data there is likely to be outliers.

In [ ]:
# plt.hist('generated', data=df[df.employ_from == 'North America'], label= 'NA')
# plt.hist('generated', data=df[df.employ_from == 'British Isles'], label= 'BI')
# plt.legend()
# plt.ylabel('Quantities of order')
# plt.xlabel('Price of order')
# plt.savefig('hist money')

There seems to be a few outliers. Lets take care of the egregious ones i.e. anything above 11K

In [ ]:
df.generated.sort_values(ascending=False).head(10)

In [ ]:
to_drop = [617,782,733,124,176,569,641,169]
df.drop(index=to_drop,inplace=True)

In [ ]:
#make two new dfs. One that contains all the employees that are employed out of British Isles and a group from North America
#Then describe them both

## 2-2 Testing

### Mann–Whitney U test

Mann–Whitney U test is a nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample.

Unlike the t-test it does not require the assumption of normal distributions. It is nearly as efficient as the t-test on normal distributions.

Mann-Whitney assumptions:  
1) The dependent variable should be measured on an ordinal scale or a continuous scale.  
2) The independent variable should be two independent, categorical groups.  
3) Observations should be independent. In other words, there should be no relationship between the two groups or within each group.  
4) Observations are not normally distributed. However, they should follow the same shape (i.e. both are bell-shaped and skewed left).  


All assumptions are met within our data. So finally, refining our hypothesis: 

$H_{o} =$ There is no statistical difference between the meadian revenues generated by the North American office and the British Island office.  
$H_{a} =$ There is a statistical difference between the meadian revenues generated by the North American office and the British Island office.  
$\alpha = 0.05$

In [ ]:
merica['generated'].hist(label='American office')
brit['generated'].hist(label='British Isle office')
plt.title('Histogram of money generate by based on office location')
plt.xlabel('Quantity of money')
plt.ylabel('Orders count')
plt.legend()
plt.show()
#plt.savefig('hist of money gen')

In [ ]:
mannwhitneyu(brit.generated, merica.generated, alternative='two-sided')

#### Independent T-test

Like every test, this inferential statistic test has assumptions. The assumptions that the data must meet in order for the test results to be valid are:  

Independent T-test Assumptions
- The samples are independently and randomly drawn
- The distribution of the residuals between the two groups should follow the normal distribution
- The variances between the two groups are equal

The first assumption is met. The second assumption can be suspended given a large enough sample size, therefore independent t-test remains quite robust for violations of normality. [More information](http://thestatsgeek.com/2013/09/28/the-t-test-and-robustness-to-non-normality/) on this topic has been provided.

Keep in mind that as a parametric test, the independent t-test delivers best and most reliable results if both groups are normally distributed. Reliability decreases for skewed distributions.

In [ ]:
ttest_ind(brit.generated, merica.generated)

### 2-3 Results

- Failed to reject the null hypothesis. There is no statistical difference between the median revenues generated by the North American office and the British Island office 
- (p-value > $\alpha$ :: 0.3749 > 0.05)  
- Further testing: choosing a different metric and test out hypothesis again.
___
___

# 3) Does having 'manager' in your title effect the amount a customer orders?


### 3-1 Inital Analysis
The customers that we do business with, there are different tiers of titles that they use. I want to know if a manager typically orders more than other customers.

$H_{o} =$ Having manager in your title, the amount a customer orders doesn't change  
$H_{a} =$ Having manager in your title, the amount a customer orders changes  
$\alpha = 0.05$

In [ ]:
get_column_info('Customer')

In [ ]:
q = '''SELECT c.CompanyName, c.ContactTitle, COUNT(*) AS orders_placed \
FROM OrderDetail od \
LEFT JOIN [Order] o ON od.OrderId = o.Id \
LEFT JOIN Customer c ON c.Id = o.CustomerId \
GROUP BY c.CompanyName'''

df = pd.read_sql_query(q, engine)
df.head(10)

In [ ]:
df.orders_placed.sum()

Good, we have all of the orders placed. There seems be a few ghost orders, someone placing some personal orders, or some testing data. Nevertheless, I will need to remove them.

In [ ]:
df.drop(index=0, inplace=True)
df.describe()

Now encoding manager title vs no mananger title.

In [ ]:
df['is_mgmt'] = np.where(df.ContactTitle.str.contains('anager'),1,0) #also could have used .lower() to include the full word

In [ ]:
df.head(10)

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='orders_placed',x='is_mgmt', data=df)
plt.title('Violin plots: comparing the distribution of management vs non-management')
plt.xlabel('Is management')
plt.ylabel('Number of items in order');

In [ ]:
df.groupby('is_mgmt').orders_placed.idxmax(),df.groupby('is_mgmt').orders_placed.max()

In [ ]:
df.drop(index=[66,19], inplace=True)
#df.drop(index=[57], inplace=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.violinplot(y='orders_placed',x='is_mgmt', data=df)
plt.title('Violin plots: comparing the distribution of management vs non-management')
plt.xlabel('Is management')
plt.ylabel('Number of items in order');

## 3-2 Testing

I will perform both the independent T-test as well as Mann-Whitney U test.  

See section 2-2 for supporting documentation

In [ ]:
mgmt_yes = df.orders_placed[df.is_mgmt == 1]
mgmt_no = df.orders_placed[df.is_mgmt == 0]
ttest_ind(mgmt_yes, mgmt_no)

In [ ]:
mannwhitneyu(mgmt_no, mgmt_yes)

## 3-3 Results

- Failed to reject the null hypothesis. As a customer having manager in your title, the quantity ordered _doesn't signifgantly change_ compared to customers without that title  
- p-value > $\alpha$ :: 0.2241 > 0.05
- Further testing: change the criteria in which the customers are grouped. Include 'owner' title with the 'manager' title and compare results.
___
___

RESOUCES:

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3116565/

https://pythonfordatascience.org/